# 04. Chains e Sequências

Já vimos chains simples com `|`. Agora vamos criar fluxos mais complexos, onde a saída de uma chain serve de entrada para outra, e execução paralela.

**Objetivos:**
- Criar chains sequenciais.
- Usar `RunnableParallel` para executar tarefas ao mesmo tempo.

In [ ]:
!pip install -qU langchain langchain-openai langchain-community python-dotenv

In [ ]:
import os
from google.colab import userdata
import getpass

try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Digite sua OpenAI API Key: ")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo")

## 1. Chain Sequencial

Imagine que queremos:
1. Gerar uma sinopse de um filme dado o título.
2. Escrever uma crítica baseada nessa sinopse de filme.

Isso são duas chains conectadas.

In [ ]:
# Chain 1: Sinopse
prompt_sinopse = ChatPromptTemplate.from_template("Escreva uma sinopse breve para um filme chamado {titulo}.")
chain_sinopse = prompt_sinopse | llm | StrOutputParser()

# Chain 2: Crítica
# Nota: o input dessa chain será a sinopse gerada anteriormente
prompt_critica = ChatPromptTemplate.from_template("Escreva uma crítica de cinema para a seguinte sinopse: {sinopse}.")
chain_critica = prompt_critica | llm | StrOutputParser()

# Encadeando tudo
# Usamos um dicionário lambda ou chain pura se o input bater
# Aqui, a chain_sinopse retorna uma string. Precisamos passá-la como 'sinopse' para a chain_critica.
from langchain_core.runnables import RunnablePassthrough

chain_completa = (
    {"sinopse": chain_sinopse} 
    | chain_critica
)

print(chain_completa.invoke({"titulo": "As Aventuras do Programador Python"}))

## 2. Execução Paralela (`RunnableParallel`)

Às vezes queremos rodar duas coisas ao mesmo tempo com o mesmo input. Ex: Dado um tema, escrever um poema E uma piada.

In [ ]:
from langchain_core.runnables import RunnableParallel

chain_poema = ChatPromptTemplate.from_template("Escreva um poema curto sobre {tema}.") | llm | StrOutputParser()
chain_piada = ChatPromptTemplate.from_template("Escreva uma piada sobre {tema}.") | llm | StrOutputParser()

mapa_combinado = RunnableParallel(poema=chain_poema, piada=chain_piada)

resultado = mapa_combinado.invoke({"tema": "Inteligência Artificial"})

print("=== POEMA ===")
print(resultado['poema'])
print("\n=== PIADA ===")
print(resultado['piada'])

## Conclusão

Vimos como compor chains sequencialmente e paralelamente.

No próximo notebook, começaremos a construir nosso sistema RAG (Retrieval Augmented Generation), aprendendo a carregar e processar documentos.